## Feature extraction 

In [1]:
from jl_song_data import SongData 

In [2]:
data = SongData('../data/chords_clean_1_2_3.csv')

In [5]:
df = data.df

In [6]:
from jl_feature_extractor import FeatureExtractor    

feature_extractor = FeatureExtractor()

In [7]:
df['artist'] = df['url'].apply(feature_extractor.extract_pretty_artist)

In [8]:
df

,url,name,decade,genre,chords,uuid,artist
0,https://tabs.ultimate-guitar.com/tab/1055161,Time To Say Goodbye Con Te Partirò,1990s,Pop%%Classical%%Pop,"['G', 'D', 'Em', 'C', 'G', 'D', 'Em', 'C', 'G'...",d42333bf-e925-4ad1-a4b9-bfe2e2df209e,Unknown
1,https://tabs.ultimate-guitar.com/tab/1060259,El Mañana (ver 3),2000s,Electronic%%Pop,"['Am', 'Em/G', 'F', 'Em', 'Bm', 'Dm', 'Am', 'E...",d6123fcf-d023-4425-8644-0a04afd88ebf,Unknown
2,https://tabs.ultimate-guitar.com/tab/1061244,Balladen Om Herr Fredrik Åkare Och Den Söta Fr...,1960s,Jazz,"['Dm', 'E', 'Am', 'Dm', 'G7', 'C', 'E7', 'Am',...",8bc3d690-78cd-4655-add3-062e7f9df1a7,Unknown
3,https://tabs.ultimate-guitar.com/tab/10cc/drea...,Dreadlock Holiday,1970s,Pop,"['Em', 'D', 'C', 'Em', 'Am', 'Em', 'Am', 'Em',...",40d74cf2-a026-4497-8b7a-08cd6527f009,10cc
4,https://tabs.ultimate-guitar.com/tab/10cc/im_n...,Im Not In Love,1970s,Pop%%Pop%%Pop,"['F#m7/B', 'B6', 'F#m7/B', 'B6', 'F#m7/B', 'B6...",277077be-828c-46ab-93c0-5ed82f514e3b,10cc
...,...,...,...,...,...,...,...
8939,https://tabs.ultimate-guitar.com/tab/ziggy_mar...,Lighthouse,2010s,Reggae,"['C', 'F', 'C', 'G', 'C', 'C', 'F', 'C', 'G', ...",fda4b7b8-e3bb-4c38-b5a7-73762c8c8261,Ziggy Marley
8940,https://tabs.ultimate-guitar.com/tab/ziggy_mar...,Personal Revolution,2010s,Reggae,"['Bm', 'A', 'Bm', 'A', 'Bm', 'A', 'Em', 'G', '...",089f0438-4e4d-44dd-83e5-5dd929753f0f,Ziggy Marley
8941,https://tabs.ultimate-guitar.com/tab/ziggy_mar...,True To Myself,2000s,Reggae%%Reggae,"['A', 'E', 'Bm', 'D', 'A', 'E', 'Bm', 'D', 'A'...",c0d75c52-ae53-4b47-bc8e-a723ba542d00,Ziggy Marley
8942,https://tabs.ultimate-guitar.com/tab/ziggy_mar...,True To Myself (ver 2),2000s,Reggae,"['A', 'E', 'Bm', 'A', 'E', 'Bm', 'A', 'E', 'Bm...",29047e58-5391-4875-ab94-3a5a7fdbcd0e,Ziggy Marley


In [17]:
by_artist = df.groupby('artist')['url'].count().reset_index()
by_artist

,artist,url
0,10cc,3
1,2pac,1
2,3 Doors Down,2
3,3oh3,1
4,4 Non Blondes,1
...,...,...
1406,Zaz,1
1407,Zazie,1
1408,Zedd,1
1409,Zero 7,3


In [25]:
by_artist.sort_values(['url'], ascending=[0]).head(10)

,artist,url
390,Elvis Presley,132
645,Johnny Cash,118
457,Frank Sinatra,98
1350,Unknown,97
843,Michael Jackson,93
1375,Weird Al Yankovic,87
1184,The Beatles,73
19,Adele,69
1314,Tom Waits,69
532,Hillsong United,65
